<a href="https://colab.research.google.com/github/Sreelakshmim6/CE888/blob/main/SVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing all libraries required
import urllib.request
import csv
import numpy as np
import pandas as pd
import string
import nltk
import re


In [ ]:
#function to load data 
def import_data(task,tag):
  l = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/{tag}.txt"
  with urllib.request.urlopen(l) as f:   
    html_1 = f.read().decode('utf-8').split("\n")
  t=pd.DataFrame(html_1)
  #t = t[~(t[0]=='')]
  return t

# HATE DATASET

In [ ]:
#Loading Train,Test and Validation data
Hate_Train_labels_df=import_data("hate","train_labels")
Hate_Train_text_df=import_data("hate","train_text")

Hate_Test_label_df=import_data("hate","test_labels")
Hate_Test_text_df=import_data("hate","test_text")
Hate_Train_labels_df.shape,Hate_Train_text_df.shape,Hate_Test_label_df.shape,Hate_Test_text_df.shape

((9001, 1), (9001, 1), (2971, 1), (2971, 1))

In [ ]:
hate_val_text=import_data("hate","val_text")
hate_val_labels=import_data("hate","val_labels")

In [ ]:
#checking the last elements of the dataframe
print(Hate_Train_text_df[0][9000])
print(Hate_Train_labels_df[0][9000])


Preprocessing

In [ ]:
pip install unicode # have to be in a separate line


In [ ]:
pip install emoji --upgrade # have to be in a separate line


Requirement already up-to-date: emoji in /usr/local/lib/python3.7/dist-packages (1.2.0)


In [ ]:
from emoji import UNICODE_EMOJI
import emoji
nltk.download('wordnet')

# function to remove "@user" as this is not required 
user="@user"
def remove_user(text):
    return " ".join([word for word in str(text).split() if word not in user])
#-----------------------------------------------------------------------------------------------
nltk.download("stopwords") #downloading stopwords in english and saving them in a varible 
from nltk.corpus import stopwords
#function to remove stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
#-----------------------------------------------------------------------------------------------
#function to remove punctuations and other symbols
PUNCT_TO_REMOVE ="/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
#-----------------------------------------------------------------------------------------------
#function to perform lemmatization using WordNetLemmatizer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
#-----------------------------------------------------------------------------------------------


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#function to call all preprocessing steps
def preprocess_data(df):
  df=df.str.lower()
  df=df.apply(lambda x: emoji.demojize(x)) #converting emojis to text
  df=df.apply(lambda x: re.sub(r"http:\S+",'',x)) #removing urls
  df=df.apply(lambda text: remove_user(text)) # removing @user
  df=df.apply(lambda text: remove_punctuation(text)) # removing punctuation
  df=df.apply(lambda text: remove_stopwords(text)) # removing stopwords
  df=df.apply(lambda text: lemmatize_words(text)) # word lemmatization
  return df

In [ ]:
#preprocessing data
clean_test=preprocess_data(Hate_Test_text_df[0])
clean_train=preprocess_data(Hate_Train_text_df[0])

In [ ]:
# removing the last empty cell for train and test texts
clean_train=clean_train[0:9000]
clean_test=clean_test[0:2970]

In [ ]:
#required libraries
import nltk
import random
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#vectorizing text using CountVectorizer and ngram
count_vect = CountVectorizer(ngram_range=(1,2)) #in scikit-learn
vect = count_vect.fit(clean_train) 
CV_train_xx=vect.transform(clean_train)
CV_test_xx=vect.transform(clean_test)

In [ ]:
# removing the last empty cell for train labels
train_Y=Hate_Train_labels_df[0].to_list()
train_Y=train_Y[0:9000]

In [ ]:
set(train_Y)

{'0', '1'}

In [ ]:
#importing SVC model
from sklearn.svm import SVC
svc=SVC(kernel= 'linear')
#fitting the model
svc.fit(CV_train_xx, train_Y)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
#predicting output for training data
pred=svc.predict(CV_test_xx)


In [ ]:
# removing the last empty cell for test labels

y_true=Hate_Test_label_df[0].to_list()
y_true=y_true[0:2970]

In [ ]:
len(y_true)

2970

Model Evalluation

In [ ]:
print('Accuracy = ',accuracy_score(pred,y_true))
print('Precision = ',precision_score(y_true, pred,average='weighted'))
print('Recall = ',recall_score(y_true, pred,average='weighted'))
print('F1_Score = ',f1_score(y_true, pred,average='macro')) # output : 0.4974439521615913
print('********Confusion Matrix********')
print(confusion_matrix(pred, y_true))

Accuracy =  0.5158249158249159
Precision =  0.601246816060096
Recall =  0.5158249158249159
F1_Score =  0.4974439521615913
********Confusion Matrix********
[[ 482  202]
 [1236 1050]]


HYPERPARAMETER TUNING USING GRIDSEARCH CV

In [155]:
# Parameters for tuning
parameters = [{'kernel': ['rbf','poly','linear'], 'gamma': [0.1,0.01,0.001],'C': [0.1,1, 10, 100, 1000]}]
print("Tuning hyper-parameters")
svc = GridSearchCV(SVC(), parameters, cv = 5)
svc.fit(CV_train_xx, train_Y)

# Checking the best values for all parameters
print("Best Parameters are: \n")
svc.best_params_

Tuning hyper-parameters
Best Parameters are: 



{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [ ]:
# Parameters for tuning
parameters = [{'kernel': ['rbf','poly'], 'gamma': [1,0.1,0.01],'C': [0.1,1, 10, 100]}]
print("Tuning hyper-parameters")
svc = GridSearchCV(SVC(), parameters, cv = 5)
svc.fit(CV_train_xx, train_Y)

# Checking the best values for all parameters
print("Best Parameters are: \n")
svc.best_params_

Tuning hyper-parameters
Best Parameters are: 



{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [ ]:
#calling the model with tuned parameters
svc=SVC(kernel='linear', C=10, gamma=0.001)
svc.fit(CV_train_xx, train_Y)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1e-09, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
#predicting outputs for test data
pred=svc.predict(CV_test_xx)


In [ ]:
Model Evaluation after tuning

In [ ]:
print('Accuracy = ',accuracy_score(pred,y_true))
print('Precision = ',precision_score(y_true, pred,average='weighted'))
print('Recall = ',recall_score(y_true, pred,average='weighted'))
print('F1_Score = ',f1_score(y_true, pred,average='macro')) # our score 0.4974439521615913 , TweetEval leaderboard score - 36.7
print('********Confusion Matrix********')
print(confusion_matrix(pred, y_true))

Accuracy =  0.5158249158249159
Precision =  0.601246816060096
Recall =  0.5158249158249159
F1_Score =  0.4974439521615913

********Confusion Matrix********
[[ 482  202]
 [1236 1050]]


Validation Set for HATE

In [ ]:
clean_val_hate=preprocess_data(hate_val_text[0]) #preprocessing val data


In [ ]:
clean_val_hate=clean_val_hate.to_list() #converting to a list
CV_val_xx=vect.transform(clean_val_hate) # vectorizing the data


In [ ]:
pred_val_hate=svc.predict(CV_val_xx) #predicting output


In [ ]:
print('F1_Score = ',f1_score(hate_val_labels, pred_val_hate,average='macro')) #output : 0.3973804892036464


F1_Score =  0.3973804892036464


# OFFENSIVE

In [ ]:
#Loading Train,Test and Validation data

Off_Train_labels_df=import_data("offensive","train_labels")
Off_Train_text_df=import_data("offensive","train_text")

Off_Test_label_df=import_data("offensive","test_labels")
Off_Test_text_df=import_data("offensive","test_text")
labels=import_data("offensive","mapping")

off_val_text=import_data("offensive","val_text")
off_val_labels=import_data("offensive","val_labels")

In [ ]:
#preprocessing the data
clean_test_off=preprocess_data(Off_Test_text_df[0])
clean_train_off=preprocess_data(Off_Train_text_df[0])

In [ ]:
#converting data to a list format
x_train_off=clean_train_off.tolist()
y_train_off=Off_Train_labels_df[0].tolist()
x_test_off=clean_test_off.tolist()
y_test_off=Off_Test_label_df[0].tolist()

In [ ]:
#removing the last elements of the dataframe as it is empty
x_train_off=x_train_off[:11916]
y_train_off=y_train_off[:11916]
x_test_off=x_test_off[:860]
y_test_off=y_test_off[:860]

In [ ]:
#Vectorizing data
count_vect = CountVectorizer(ngram_range=(1,2)) #in scikit-learn
vect = count_vect.fit(x_train_off) 
CV_train_xx=vect.transform(x_train_off)
CV_test_xx=vect.transform(x_test_off)

In [ ]:
#importing SVC model from sklearn
from sklearn.svm import SVC
svc=SVC(kernel= 'linear')
svc.fit(CV_train_xx, y_train_off)# fitting the model


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred_off=svc.predict(CV_test_xx) #predicting output for test data


Model Evaluation

In [ ]:
print('Accuracy = ',accuracy_score(pred_off,y_test_off))
print('Precision = ',precision_score(y_test_off, pred_off,average='weighted'))
print('Recall = ',recall_score(y_test_off, pred_off,average='weighted'))
print('F1_Score = ',f1_score(y_test_off, pred_off,average='macro')) #0.7167177563499108 / leaderboard score for SVM - 52.3
print()
print('********Confusion Matrix********')
print(confusion_matrix(pred_off, y_test_off))

Accuracy =  0.7872093023255814
Precision =  0.7771020713998026
Recall =  0.7872093023255814
F1_Score =  0.7167177563499108

********Confusion Matrix********
[[553 116]
 [ 67 124]]


Hyperparameter tuning 

In [ ]:
# Parameters for tuning
parameters = [{'kernel': ['rbf','poly','linear'], 'gamma': [1,0.1,0.01,0.001],'C': [0.1,1, 10, 100, 1000, 10000]}]
print("Tuning hyper-parameters")
svc = GridSearchCV(SVC(), parameters, cv = 5)
svc.fit(CV_train_xx, y_train_off)

# Checking the score for all parameters
print("Best Parameters are: \n")
svc.best_params_

Validation Set for OFFENSIVE

In [ ]:
clean_val_off=preprocess_data(off_val_text[0]) #preprocessing val data
clean_val_off=clean_val_off.to_list() #converting to a list
CV_val_xx=vect.transform(clean_val_off)#vectorization


In [ ]:
pred_val_off=svc.predict(CV_val_xx)#predicting output


In [ ]:
print('F1_Score = ',f1_score(off_val_labels, pred_val_off,average='macro')) #0.4839709824107536


F1_Score =  0.4839709824107536


# IRONY

In [ ]:
#loading dat
irony_Train_labels_df=import_data("irony","train_labels")
irony_Train_text_df=import_data("irony","train_text")

irony_Test_label_df=import_data("irony","test_labels")
irony_Test_text_df=import_data("irony","test_text")
labels=import_data("irony","mapping")

irony_val_text=import_data("irony","val_text")
irony_val_labels=import_data("irony","val_labels")

In [ ]:
#preprocessing data
clean_test_irony=preprocess_data(irony_Test_text_df[0])
clean_train_irony=preprocess_data(irony_Train_text_df[0])

In [ ]:
#converting to a list
x_train_irony=clean_train_irony.tolist()
y_train_irony=irony_Train_labels_df[0].tolist()
x_test_irony=clean_test_irony.tolist()
y_test_irony=irony_Test_label_df[0].tolist()


In [ ]:
#removing the last elements of the dataframe as it is empty
x_train_irony=x_train_irony[:2862]
y_train_irony=y_train_irony[:2862]
x_test_irony=x_test_irony[:784]
y_test_irony=y_test_irony[:784]
labels2=[0,1]

In [ ]:
#Vectorizing data
count_vect = CountVectorizer(ngram_range=(1,2)) #in scikit-learn
vect = count_vect.fit(x_train_irony) 
CV_train_xx=vect.transform(x_train_irony)
CV_test_xx=vect.transform(x_test_irony)

In [ ]:
svc=SVC(kernel= 'linear') #initializing the model
svc.fit(CV_train_xx, y_train_irony) #fitting the model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred_irony=svc.predict(CV_test_xx) #predicting for train data


Model Evaluation

In [ ]:
print('Accuracy = ',accuracy_score(pred_irony,y_test_irony))
print('Precision = ',precision_score(y_test_irony, pred_irony,average='weighted'))
print('Recall = ',recall_score(y_test_irony, pred_irony,average='weighted'))
print('F1_Score = ',f1_score(y_test_irony, pred_irony,average='macro')) #0.65
print()
print('********Confusion Matrix********')
print(confusion_matrix(pred_irony, y_test_irony))

Accuracy =  0.6556122448979592
Precision =  0.6714081300001663
Recall =  0.6556122448979592
F1_Score =  0.6501457725947521

********Confusion Matrix********
[[306 103]
 [167 208]]


Hyperparameter tuning

In [ ]:
# Parameters for tuning
parameters = [{'kernel': ['rbf','poly','linear'], 'gamma': [1,0.1,0.01,0.001],'C': [0.1,1, 10, 100, 1000, 10000]}]
print("Tuning hyper-parameters")
svc = GridSearchCV(SVC(), parameters, cv = 5)
svc.fit(CV_train_xx, y_train_irony)

# Checking the score for all parameters
print("Best Parameters are: \n")
svc.best_params_

Validation Set for Irony

In [ ]:
clean_val_irony=preprocess_data(irony_val_text[0]) #preprocessing val data
clean_val_irony=clean_val_irony.to_list() #converting to a list
CV_val_xx=vect.transform(clean_val_irony) #vectorization


In [ ]:
pred_val_irony=svc.predict(CV_val_xx) #predicting outputs for val data


In [ ]:
print('F1_Score = ',f1_score(irony_val_labels, pred_val_irony,average='macro')) #output is 0.406757978830516


F1_Score =  0.406757978830516
